In [1]:
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.utils import to_categorical
import os
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.callbacks import EarlyStopping
from keras.applications import VGG16, InceptionV3, ResNet50, VGG19, Xception

import sys


/home/rlee/venvp3/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
model_flag = 1

if model_flag == 1:
    vgg_conv = VGG16(weights='imagenet',
                      include_top=False,
                      input_shape=(224, 224, 3))
    vgg_conv.summary()
elif model_flag == 2:
    inc_conv = InceptionV3(weights='imagenet',
                  include_top=False,
                  input_shape=(224, 224, 3))
elif model_flag == 3:
    resnet_conv = ResNet50(weights='imagenet',
                  include_top=False,
                  input_shape=(224, 224, 3))
elif model_flag == 4:
    vgg19_conv = VGG19(weights='imagenet',
                  include_top=False,
                  input_shape=(224, 224, 3))
elif model_flag == 5:
    xcep_conv = Xception(weights='imagenet',
                      include_top=False,
                      input_shape=(299,299,3))


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [3]:
train_dir = '/home/rlee/Documents/Pneumothorax/model/mmode/train'
validation_dir = '/home/rlee/Documents/Pneumothorax/model/mmode/val'

nTrain = 1829
nVal = 457

if model_flag == 5:
    datagen = ImageDataGenerator(rescale=1./255,data_format="channels_last")
    target_sz = 299
else:
    datagen = ImageDataGenerator(rescale=1./255)
    target_sz = 224

batch_size = 32

if model_flag == 1:
    d1 = 7
    d2 = 7
    d3 = 512
    model_name = 'VGG16'
elif model_flag == 2:
    d1 = 5
    d2 = 5
    d3 = 2048
    model_name = 'InceptionV3'
elif model_flag == 3:
    d1 = 1
    d2 = 1
    d3 = 2048
    model_name = 'Resnet50'
elif model_flag == 4:
    d1 = 7
    d2 = 7
    d3 = 512
    model_name = 'VGG19'
elif model_flag == 5:
    d1 = 10
    d2 = 10
    d3 = 2048
    model_name = 'Xception'

train_features = np.zeros(shape=(nTrain, d1, d2, d3))

train_labels = np.zeros(shape=(nTrain,2))

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(target_sz, target_sz),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True)

print('Obtaining training data')
i = 0
for inputs_batch, labels_batch in train_generator:
    if i%50 == 0:
        print(i)
    if model_flag == 1:
        features_batch = vgg_conv.predict(inputs_batch)
    elif model_flag == 2:
        features_batch = inc_conv.predict(inputs_batch)
    elif model_flag == 3:
        features_batch = resnet_conv.predict(inputs_batch)
    elif model_flag == 4:
        features_batch = vgg19_conv.predict(inputs_batch)        
    elif model_flag == 5:
        features_batch = xcep_conv.predict(inputs_batch) 
    
    train_features[i * batch_size : (i + 1) * batch_size] = features_batch
    train_labels[i * batch_size : (i + 1) * batch_size] = labels_batch
    i += 1
    if i * batch_size >= nTrain:
        break


train_features = np.reshape(train_features, (nTrain, d1 * d2 * d3))


Found 1829 images belonging to 2 classes.
Obtaining training data
0
50


In [4]:
validation_features = np.zeros(shape=(nVal, d1, d2, d3))

validation_labels = np.zeros(shape=(nVal,2))

validation_generator = datagen.flow_from_directory(
    validation_dir,
    target_size=(target_sz, target_sz),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False)

print('Obtaining validation data')
i = 0
for inputs_batch, labels_batch in validation_generator:
    if model_flag == 1:
        features_batch = vgg_conv.predict(inputs_batch)
    elif model_flag == 2:
        features_batch = inc_conv.predict(inputs_batch)
    elif model_flag == 3:
        features_batch = resnet_conv.predict(inputs_batch)    
    elif model_flag == 4:
        features_batch = vgg19_conv.predict(inputs_batch)        
    elif model_flag == 5:
        features_batch = xcep_conv.predict(inputs_batch) 
    
    validation_features[i * batch_size : (i + 1) * batch_size] = features_batch
    validation_labels[i * batch_size : (i + 1) * batch_size] = labels_batch
    i += 1
    if i * batch_size >= nVal:
        break

validation_features = np.reshape(validation_features, (nVal, d1 * d2 * d3))


Found 457 images belonging to 2 classes.
Obtaining validation data


In [47]:
from keras import models
from keras import layers
from keras import optimizers, initializers, regularizers, callbacks

activ = 'sigmoid'
loss = 'categorical_crossentropy'
optim = 1
init = initializers.RandomNormal()
bnorm_flag = True

In [48]:
class my_model():
    def __init__(self, drpout, init, activ, bnorm_flag, lrt, loss, optim, decay_rt, decay_steps, 
                 batch_size, n_epochs, train_X, train_y, val_X, val_y, test_X, test_y):
        self.drpout = drpout
        self.init = init
        self.activ = activ
        self.bnorm_flag = bnorm_flag
        self.lrt = lrt
        self.loss = loss
        self.optim = optim
        self.decay_rt = decay_rt
        self.decay_steps = np.max([np.round(decay_steps*n_epochs), 1])
        self.batch_size = batch_size
        self.n_epochs = n_epochs
        self.train_X = train_X
        self.train_y = train_y
        self.val_X = val_X
        self.val_y = val_y        
        self.test_X = test_X
        self.test_y = test_y
        self.model = self.set_up_model()
        
    def set_up_model(self):
        
        model = models.Sequential()
        
        '''
        if self.init == 0:
            init = initializers.RandomNormal()
        else:
            init = initializers.glorot_normal()
        
        activ = 'relu'
        if self.activ == 1:
            activ = 'relu'
        elif self.activ == 2:
            activ = 'tanh'
        elif self.activ == 3:
            activ = 'sigmoid'
            
        if self.loss == 1:
            loss = 'categorical_crossentropy'
        elif self.loss == 2:
            loss = 'mean_squared_error'
        elif self.loss == 3:
            loss = 'kullback_leibler_divergence'
        elif self.loss == 4:
            loss = 'categorical_hinge'
        '''
        
        model.add(layers.Dense(d3, activation=self.activ, input_dim=d1 * d2 * d3, kernel_initializer=self.init))#, kernel_regularizer=regularizers.l2(0.01)))
        if self.bnorm_flag:
            model.add(layers.BatchNormalization())
        model.add(layers.Dropout(self.drpout))
        model.add(layers.Dense(2, activation='softmax', kernel_initializer=self.init))#, kernel_regularizer=regularizers.l2(0.01)))
        
        optim = optimizers.RMSprop(lr=self.lrt)
        if self.optim == 1:
            optim = optimizers.RMSprop(lr=self.lrt)
        elif self.optim == 2:
            optim = optimizers.Adam(lr=self.lrt)
        elif self.optim == 3:
            optim = optimizers.SGD(lr=self.lrt)
            
        model.compile(optimizer=optim, loss=loss, metrics=['acc'])

        return model
    
    def step_decay(self, epoch):
            steps = self.decay_steps#np.floor(n_epochs/3)
            lrate = self.lrt * (self.decay_rt)**np.floor(epoch/steps)
            return lrate
        
    def train_model(self):
        
        earlystop = EarlyStopping(patience=10)
        checkpointer = callbacks.ModelCheckpoint(filepath='/home/rlee/weights.hdf5', verbose=1, save_best_only=True)
        lrate=keras.callbacks.LearningRateScheduler(self.step_decay,verbose=1)
        callbacks_list = [earlystop, lrate, checkpointer]
        history = self.model.fit(self.train_X,
                self.train_y,
                epochs=self.n_epochs,
                callbacks=callbacks_list,
                batch_size=self.batch_size,
                validation_data=(self.val_X,self.val_y))
        #dscr_phrase = 'rmsprop' + str(lrt) + 'decay_' + str(decay_rt) + '_' + loss + 'glorot_normal_init'# + 'dropout_' + str(drpout)
        #model.save(model_name + '_mmode_nopreprocessing_' + dscr_phrase + '.h5')
        self.model = keras.models.load_model('/home/rlee/weights.hdf5')
        
    def model_evaluate(self):
        self.train_model()
        
        evaluation = self.model.evaluate(self.test_X, self.test_y, batch_size=self.batch_size, verbose=0)
        return evaluation

In [49]:
def run_my_model(drpout, init, activ, bnorm_flag, lrt, loss, optim, decay_rt, decay_steps, 
                 batch_size, n_epochs, train_X, train_y, val_X, val_y, test_X, test_y):
    
    model = my_model(drpout=drpout, init=init, activ=activ, bnorm_flag=bnorm_flag, lrt=lrt, loss=loss, 
                     optim=optim, decay_rt=decay_rt, decay_steps=decay_steps, batch_size=batch_size, 
                     n_epochs=n_epochs, train_X=train_X, train_y=train_y, val_X=val_X, val_y=val_y, 
                     test_X=test_X, test_y=test_y)
    model_evaluation = model.model_evaluate()
    return model_evaluation


In [50]:
# bounds for hyper-parameters in model
# the bounds dict should be in order of continuous type and then discrete type
bounds = [{'name': 'drpout', 'type': 'continuous',  'domain': (0.0, 0.8)},
          {'name': 'lrt',          'type': 'continuous',  'domain': (1e-6, 1e-3)},
          {'name': 'decay_rt',          'type': 'continuous',  'domain': (1e-6, 1.0)},
          {'name': 'decay_steps',           'type': 'discrete',    'domain': (0.1, 0.2, 0.3, 0.5)},
          {'name': 'batch_size',       'type': 'discrete',    'domain': (16, 32, 64, 128)},
          {'name': 'n_epochs',           'type': 'discrete',    'domain': (20, 30, 50, 100)}]

In [51]:
# function to optimize mnist model
def f(x):
    print(x)
    evaluation = run_my_model(drpout=float(x[:,0]), init=init, activ=activ, bnorm_flag=bnorm_flag, 
                              lrt=float(x[:,1]), loss=loss, optim=optim, decay_rt=float(x[:,2]), 
                                        decay_steps=int(x[:,3]), batch_size=int(x[:,4]), n_epochs=int(x[:,5]),
                             train_X=train_features, train_y=train_labels, val_X=validation_features,
                             val_y=validation_labels, test_X=validation_features, test_y=validation_labels)
    print("LOSS:\t{0} \t ACCURACY:\t{1}".format(evaluation[0], evaluation[1]))
    print(evaluation)
    return evaluation[0]

In [ ]:
import GPy, GPyOpt

opt_model = GPyOpt.methods.BayesianOptimization(f=f, domain=bounds)
opt_model.run_optimization(max_iter=10)


[[2.01015912e-01 3.00535730e-04 1.88362094e-01 1.00000000e-01
  6.40000000e+01 5.00000000e+01]]
Train on 1829 samples, validate on 457 samples
Epoch 1/50

Epoch 00001: LearningRateScheduler reducing learning rate to 0.00030053572971616656.
1829/1829 [==============================] - 13s 7ms/step - loss: 0.2322 - acc: 0.9120 - val_loss: 0.1710 - val_acc: 0.9278

Epoch 00001: val_loss improved from inf to 0.17096, saving model to /home/rlee/weights.hdf5
Epoch 2/50

Epoch 00002: LearningRateScheduler reducing learning rate to 5.6609539464509264e-05.
1829/1829 [==============================] - 4s 2ms/step - loss: 0.0744 - acc: 0.9727 - val_loss: 0.1417 - val_acc: 0.9387

Epoch 00002: val_loss improved from 0.17096 to 0.14171, saving model to /home/rlee/weights.hdf5
Epoch 3/50

Epoch 00003: LearningRateScheduler reducing learning rate to 1.0663091411494977e-05.
1829/1829 [==============================] - 5s 3ms/step - loss: 0.0651 - acc: 0.9765 - val_loss: 0.0993 - val_acc: 0.9562

Epoch

1829/1829 [==============================] - 4s 2ms/step - loss: 0.1170 - acc: 0.9508 - val_loss: 0.1155 - val_acc: 0.9519

Epoch 00012: val_loss improved from 0.11576 to 0.11550, saving model to /home/rlee/weights.hdf5
Epoch 13/30

Epoch 00013: LearningRateScheduler reducing learning rate to 6.11797206250439e-13.
1829/1829 [==============================] - 4s 2ms/step - loss: 0.1417 - acc: 0.9420 - val_loss: 0.1154 - val_acc: 0.9540

Epoch 00013: val_loss improved from 0.11550 to 0.11542, saving model to /home/rlee/weights.hdf5
Epoch 14/30

Epoch 00014: LearningRateScheduler reducing learning rate to 1.0684343025162149e-13.
1829/1829 [==============================] - 3s 2ms/step - loss: 0.1134 - acc: 0.9546 - val_loss: 0.1152 - val_acc: 0.9540

Epoch 00014: val_loss improved from 0.11542 to 0.11524, saving model to /home/rlee/weights.hdf5
Epoch 15/30

Epoch 00015: LearningRateScheduler reducing learning rate to 1.8658991037072778e-14.
1829/1829 [==============================] - 3s 

In [46]:
print("""
Optimized Parameters:
\t{0}:\t{1}
\t{2}:\t{3}
\t{4}:\t{5}
\t{6}:\t{7}
\t{8}:\t{9}
\t{10}:\t{11}
""".format(bounds[0]["name"],opt_model.x_opt[0],
           bounds[1]["name"],opt_model.x_opt[1],
           bounds[2]["name"],opt_model.x_opt[2],
           bounds[3]["name"],opt_model.x_opt[3],
           bounds[4]["name"],opt_model.x_opt[4],
           bounds[5]["name"],opt_model.x_opt[5],))
print("optimized loss: {0}".format(opt_model.fx_opt))


Optimized Parameters:
	drpout:	0.09117103101966963
	lrt:	0.00021736975729206076
	decay_rt:	0.95286935869545
	decay_steps:	0.3
	batch_size:	64.0
	n_epochs:	100.0

optimized loss: [0.07863146]
